In [6]:
using LinearAlgebra
using Plots
using SparseArrays

# Basic parameters
vs = 1000.0  # Wave velocity [m/s]
f = 0.5  # Central frequency of the source (in Hz)
λ = vs / f  # Wavelength

# Spatial and temporal parameters based on CFL condition
ep = 0.3  # Stability limit
dx = 0.1 * λ  # Spatial step size
dx_min = dx
dt = ep * dx / vs  # Time step size based on CFL

t_max = 20.0  # Maximum simulation time (in seconds)
x_max = 10000.0  # Maximum spatial coordinate (in meters)

# Print information
println("Spatial step size: $dx")
println("Time step size: $dt")

# Calculate number of steps
nt = round(Int, t_max / dt)
nx = round(Int, x_max / dx)
ny = nx  # Assuming square domain for 2D
println("Number of time steps: $nt")
println("Number of spatial steps: $nx, $ny")

# Initialize spatial coordinates
elem = zeros(nx,ny)

for i in 1:nx
    for j in 1:ny
        elem[i,j] = (i-1)*nx + j
    end
end


# parameters physical
ro0 = 1500    # Density [kg/m^3]
ro = elem * 0 .+ ro0          # initialize density array
young_mod = elem * 0 .+ ro .* vs .^ 2   # calculate young modulus from density and velocity

# Time axis
t = range(dt, stop=nt * dt, length=nt)

# Displacement
u = zeros(nx, ny)
uold = zeros(nx, ny)
unew = zeros(nx, ny)

# Source term (simple Ricker wavelet)
function ricker_wavelet(t, f, tau=2.0)
    # A = (1 - 2 * π^2 * f^2 * (t - tau)^2)
    # wavelet = A * exp(-π^2 * f^2 * (t - tau)^2)
    wavelet = sin(2 * π * f * (t - tau)) * exp(-π^2 * f^2 * (t - tau)^2)
    return wavelet
end

function bilinear_basis(xi, yi)
  return [(1 - xi) * (1 - yi) / 4, (1 + xi) * (1 - yi) / 4, (1 + xi) * (1 + yi) / 4, (1 - xi) * (1 + yi) / 4]
end

function jacobian(x1, x2, y1, y2)
  # Calculate basis function values at each point
  b11 = (1 - x1 / (x2 - x1)) * (1 - y1 / (y2 - y1)) / 4
  b12 = (1 + x1 / (x2 - x1)) * (1 - y1 / (y2 - y1)) / 4
  b21 = (1 + x1 / (x2 - x1)) * (1 + y1 / (y2 - y1)) / 4
  b22 = (1 - x1 / (x2 - x1)) * (1 + y1 / (y2 - y1)) / 4

  # Define Jacobian matrix
  J = zeros(2, 2)
  J[1, 1] = (b22 - b12) / (x2 - x1)
  J[1, 2] = (b21 - b11) / (y2 - y1)
  J[2, 1] = (b12 - b22) / (x2 - x1)
  J[2, 2] = (b11 - b21) / (y2 - y1)

  return J
end



Spatial step size: 200.0
Time step size: 0.06
Number of time steps: 333
Number of spatial steps: 50, 50


bilinear_basis (generic function with 1 method)